# Avaliação do Dataset Iris

Este notebook apresenta a análise completa e visualizações dos resultados obtidos com o dataset Iris, incluindo:
- Aprendizado Supervisionado
- Aprendizado Não Supervisionado
- Métricas de Avaliação
- Comparações entre Modelos

In [ ]:
# Importação de Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    silhouette_score, davies_bouldin_score, confusion_matrix
)
import warnings
warnings.filterwarnings('ignore')

# Configuração de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Bibliotecas importadas com sucesso")

## 1. Carregamento e Exploração dos Dados

In [ ]:
# Carregar dataset Iris
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

# Criar DataFrame
df = pd.DataFrame(X, columns=feature_names)
df['species'] = [target_names[i] for i in y]

print(f"Dataset Iris")
print(f"Amostras: {len(df)}")
print(f"Features: {len(feature_names)}")
print(f"Classes: {list(target_names)}")
print(f"\nDistribuição das Classes:")
print(df['species'].value_counts())

df.head(10)

In [ ]:
# Visualização exploratória
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Pairplot das features
for idx, feature in enumerate(feature_names[:4]):
    ax = axes[idx // 2, idx % 2]
    for species in target_names:
        data = df[df['species'] == species][feature]
        ax.hist(data, alpha=0.6, label=species, bins=15)
    ax.set_xlabel(feature)
    ax.set_ylabel('Frequência')
    ax.set_title(f'Distribuição: {feature}')
    ax.legend()

plt.tight_layout()
plt.savefig('iris_exploratory.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Análise exploratória concluída")

## 2. Aprendizado Supervisionado - Classificação

In [ ]:
# Preparar dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Normalizar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelos
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42, n_estimators=100),
    "SVM": SVC(random_state=42),
    "Naive Bayes": GaussianNB(),
    "K-NN": KNeighborsClassifier(n_neighbors=5),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42)
}

results_supervised = []

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    results_supervised.append({
        'Modelo': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred, average='weighted')
    })

df_supervised = pd.DataFrame(results_supervised)
print("\nResultados - Aprendizado Supervisionado:")
print(df_supervised.round(4).to_string(index=False))

df_supervised.to_csv('iris_supervised_results.csv', index=False)
print("\n✓ Resultados salvos em: iris_supervised_results.csv")

In [ ]:
# Visualização dos resultados supervisionados
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    bars = ax.bar(df_supervised['Modelo'], df_supervised[metric], color='skyblue', edgecolor='black')
    
    # Linha de meta 70%
    ax.axhline(y=0.70, color='red', linestyle='--', linewidth=2, label='Meta 70%')
    
    # Colorir barras acima da meta
    for bar, value in zip(bars, df_supervised[metric]):
        if value >= 0.70:
            bar.set_color('green')
            bar.set_alpha(0.7)
    
    # Adicionar valores nas barras
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    ax.set_ylabel(metric, fontsize=12, fontweight='bold')
    ax.set_title(f'{metric} por Modelo - Iris Dataset', fontsize=14, fontweight='bold')
    ax.set_xticklabels(df_supervised['Modelo'], rotation=45, ha='right')
    ax.set_ylim(0, 1.1)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('iris_supervised_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualização supervisionada salva")

## 3. Aprendizado Não Supervisionado - Clustering

In [ ]:
# K-Means Clustering
X_scaled = StandardScaler().fit_transform(X)
kmeans = KMeans(n_clusters=3, random_state=42)
clusters_kmeans = kmeans.fit_predict(X_scaled)

# Métricas
silhouette_kmeans = silhouette_score(X_scaled, clusters_kmeans)
davies_bouldin_kmeans = davies_bouldin_score(X_scaled, clusters_kmeans)

print(f"K-Means Clustering:")
print(f"  Silhouette Score: {silhouette_kmeans:.4f}")
print(f"  Davies-Bouldin Index: {davies_bouldin_kmeans:.4f}")

# Hierarchical Clustering
hierarchical = AgglomerativeClustering(n_clusters=3, linkage='average')
clusters_hier = hierarchical.fit_predict(X_scaled)

silhouette_hier = silhouette_score(X_scaled, clusters_hier)
davies_bouldin_hier = davies_bouldin_score(X_scaled, clusters_hier)

print(f"\nHierarchical Clustering:")
print(f"  Silhouette Score: {silhouette_hier:.4f}")
print(f"  Davies-Bouldin Index: {davies_bouldin_hier:.4f}")

# Resultados
results_unsupervised = pd.DataFrame({
    'Método': ['K-Means', 'Hierarchical'],
    'Silhouette Score': [silhouette_kmeans, silhouette_hier],
    'Davies-Bouldin Index': [davies_bouldin_kmeans, davies_bouldin_hier]
})

results_unsupervised.to_csv('iris_unsupervised_results.csv', index=False)
print("\n✓ Resultados não supervisionados salvos")

In [ ]:
# Visualização do clustering
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Real labels
scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', s=50, alpha=0.7, edgecolors='k')
axes[0].set_title('Classes Reais - Iris', fontsize=14, fontweight='bold')
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variância)')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variância)')
plt.colorbar(scatter1, ax=axes[0])

# K-Means
scatter2 = axes[1].scatter(X_pca[:, 0], X_pca[:, 1], c=clusters_kmeans, cmap='viridis', s=50, alpha=0.7, edgecolors='k')
axes[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
                c='red', marker='X', s=300, edgecolors='black', linewidths=2, label='Centróides')
axes[1].set_title(f'K-Means\n(Silhouette: {silhouette_kmeans:.3f})', fontsize=14, fontweight='bold')
axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variância)')
axes[1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variância)')
axes[1].legend()
plt.colorbar(scatter2, ax=axes[1])

# Hierarchical
scatter3 = axes[2].scatter(X_pca[:, 0], X_pca[:, 1], c=clusters_hier, cmap='viridis', s=50, alpha=0.7, edgecolors='k')
axes[2].set_title(f'Hierarchical Clustering\n(Silhouette: {silhouette_hier:.3f})', fontsize=14, fontweight='bold')
axes[2].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variância)')
axes[2].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variância)')
plt.colorbar(scatter3, ax=axes[2])

plt.tight_layout()
plt.savefig('iris_clustering_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualização de clustering salva")

## 4. Redução de Dimensionalidade - PCA

In [ ]:
# PCA completo
pca_full = PCA()
pca_full.fit(X_scaled)

# Variância explicada
var_exp = pca_full.explained_variance_ratio_
var_cum = np.cumsum(var_exp)

print("Análise de Componentes Principais (PCA):")
for i, (var, cum) in enumerate(zip(var_exp, var_cum)):
    print(f"  PC{i+1}: {var:.4f} ({var*100:.2f}%) | Acumulada: {cum:.4f} ({cum*100:.2f}%)")

n_components_95 = (var_cum >= 0.95).argmax() + 1
print(f"\nComponentes para 95% variância: {n_components_95}")

# Visualização
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Variância por componente
axes[0].bar(range(1, len(var_exp)+1), var_exp, alpha=0.7, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Componente Principal', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Variância Explicada', fontsize=12, fontweight='bold')
axes[0].set_title('Variância Explicada por Componente - Iris', fontsize=14, fontweight='bold')
axes[0].set_xticks(range(1, len(var_exp)+1))
axes[0].grid(axis='y', alpha=0.3)

# Variância acumulada
axes[1].plot(range(1, len(var_cum)+1), var_cum, marker='o', linewidth=2, markersize=8, color='darkorange')
axes[1].axhline(y=0.95, color='red', linestyle='--', linewidth=2, label='95% Variância')
axes[1].axvline(x=n_components_95, color='green', linestyle='--', linewidth=2, label=f'{n_components_95} Componentes')
axes[1].set_xlabel('Número de Componentes', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Variância Acumulada', fontsize=12, fontweight='bold')
axes[1].set_title('Variância Acumulada - Iris', fontsize=14, fontweight='bold')
axes[1].set_xticks(range(1, len(var_cum)+1))
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('iris_pca_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Análise PCA salva")

## 5. Resumo e Comparação Final

In [ ]:
# Resumo final
print("="*60)
print("RESUMO DA AVALIAÇÃO - DATASET IRIS")
print("="*60)

print("\n1. APRENDIZADO SUPERVISIONADO:")
best_supervised = df_supervised.loc[df_supervised['Accuracy'].idxmax()]
print(f"   Melhor Modelo: {best_supervised['Modelo']}")
print(f"   Accuracy: {best_supervised['Accuracy']:.4f}")
print(f"   F1-Score: {best_supervised['F1-Score']:.4f}")
above_70 = (df_supervised['Accuracy'] >= 0.70).sum()
print(f"   Modelos >= 70%: {above_70}/{len(df_supervised)}")

print("\n2. APRENDIZADO NÃO SUPERVISIONADO:")
print(f"   K-Means Silhouette: {silhouette_kmeans:.4f}")
print(f"   Hierarchical Silhouette: {silhouette_hier:.4f}")
print(f"   Melhor método: {'K-Means' if silhouette_kmeans > silhouette_hier else 'Hierarchical'}")

print("\n3. REDUÇÃO DE DIMENSIONALIDADE:")
print(f"   Componentes para 95% variância: {n_components_95}")
print(f"   PC1 + PC2 explicam: {var_cum[1]*100:.2f}% da variância")

print("\n4. ARQUIVOS GERADOS:")
print("   ✓ iris_exploratory.png")
print("   ✓ iris_supervised_metrics.png")
print("   ✓ iris_clustering_comparison.png")
print("   ✓ iris_pca_analysis.png")
print("   ✓ iris_supervised_results.csv")
print("   ✓ iris_unsupervised_results.csv")

print("\n" + "="*60)
print("AVALIAÇÃO CONCLUÍDA COM SUCESSO")
print("="*60)